In [1]:
from bokeh.io import output_notebook 
from bokeh.plotting import show, figure
output_notebook()
import numpy as np
import pandas as pd
from main import Atmosphere, justdoit
import astropy.units as u

Loading BokehJS ...

In [10]:
#set cloud species to condense
a = Atmosphere(['Cr','MgSiO3'],fsed=0.01)
#set the planet gravity
a.get_gravity(gravity=357.00, gravity_unit=u.Unit('cm/(s**2)'))
#set the pressure temperature profile and kzz
a.get_pt( filename='/Users/nbatalh1/Documents/codes/TheOGTC/temp.dat',skiprows=3, delim_whitespace=True, header=None, names=["ind","pressure","temperature","kz"])

#get total opacity, single scattering, asymmetry, and individual optical depths 
opd, w0, g0, opd_gas,rg= justdoit(a, directory='/Users/nbatalh1/Documents/data/eddy_optics', do_optics=False)


Overwriting some Kz values to minimum value set by kz_min


In [11]:
import pickle as pk 
opd_old, w0_old, g0_old, opd_gas_old,rg_old = pk.load(open('junk.pk','rb'))
#pk.dump([opd, w0, g0, opd_gas,rg],open('junk.pk','wb'))

In [12]:
p = figure(height=200,width=400,
    y_range=[1e2,2e-6],y_axis_type='log',x_axis_type='log',x_range=[1e-5,1e-2])
p.line(rg[:,0],a.pressure[0:-1])
p.line(rg_old[:,0],a.pressure[0:-1],color='red')
show(p)

In [13]:
p = figure(height=200,width=400,
    y_range=[1e2,2e-6],y_axis_type='log',x_range=[0,1])
p.line(g0[:,0],a.pressure[0:-1],line_dash='dashed')
p.line(g0_old[:,0],a.pressure[0:-1],color='red',line_dash='dashed')
p.line(w0[:,0],a.pressure[0:-1])
p.line(w0_old[:,0],a.pressure[0:-1],color='red')
show(p)

In [14]:
df = pd.DataFrame(index=[ i for i in range(opd.shape[0]*opd.shape[1])], columns=['lvl','w','opd','w0','g0'])
i = 0 
LVL = []
WV,OPD,WW0,GG0 =[],[],[],[]
for j in range(opd.shape[0]):
       for w in range(opd.shape[1]):
            LVL +=[j+1]
            WV+=[w+1]
            OPD+=[opd[j,w]]
            WW0+=[w0[j,w]]
            GG0+=[g0[j,w]]
df.iloc[:,0 ] = LVL
df.iloc[:,1 ] = WV
df.iloc[:,2 ] = OPD
df.iloc[:,3 ] = WW0
df.iloc[:,4 ] = GG0

In [15]:
import picaso.justplotit as jpi

In [16]:
show(jpi.plot_cld_input(nwno=196, nlayer=52,df=df,pressure=a.pressure))